In [329]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from utils.data_utils import get_datasets
from configs import load_configs
from log_tools import logger, save_results
from models import OpenSetModel, train_autoencoder
import seaborn as sns
from sklearn.metrics import roc_auc_score, balanced_accuracy_score
from eval import *

from sklearn.mixture import GaussianMixture

In [330]:
configs = load_configs("configs/config.yaml")
datasets = get_datasets(configs["data"])

In [331]:
gmm_list = []

for i in range(np.unique(datasets["init_known"]["train"][1]).shape[0]):
    gmm = GaussianMixture(n_components=3, covariance_type='full')
    gmm.fit(datasets["init_known"]["train"][0][datasets["init_known"]["train"][1] == i])
    gmm_list.append(gmm)
# open_set_model.find_best_thresholds(
#     np.concatenate([datasets["init_known"]["val"][0], datasets["emerging"]["learning"][0]]),
#     np.concatenate([datasets["init_known"]["val"][1], datasets["emerging"]["learning"][1]]),
# )

In [332]:
# eval_datasets = np.concatenate(
#     [
#         datasets["init_sources"]["test"][0][datasets["init_sources"]["test"][1] == 0],
#         datasets["emerging"]["test"][0],
#     ]
# )
# eval_labels = np.concatenate(
#     [
#         datasets["init_sources"]["test"][1][datasets["init_sources"]["test"][1] == 0],
#         # np.zeros_like(datasets["emerging"]["test"][1]),
#     ]
# )

# scores = np.zeros((eval_datasets.shape[0], len(gmm_list)))
# for i, gmm in enumerate(gmm_list):
#     scores[:, i] = gmm.score_samples(eval_datasets)

# preds = np.argmax(scores, axis=1)
# acc = balanced_accuracy_score(eval_labels, preds)
# print(f"Balanced accuracy: {round(acc * 100, 4)}")

In [340]:
configs = load_configs("configs/config.yaml")
datasets = get_datasets(configs["data"])

eval_datasets = np.concatenate(
    [
        datasets["init_known"]["test"][0][datasets["init_known"]["test"][1] == 0],
        datasets["emerging"]["test"][0],
    ]
)
eval_labels = np.concatenate(
    [
        np.ones_like(datasets["init_known"]["test"][1][datasets["init_known"]["test"][1] == 0]),
        np.zeros_like(datasets["emerging"]["test"][1]),
    ]
)

threshold_datasets = np.concatenate(
    [
        datasets["init_known"]["val"][0],
        datasets["ood"][0],
    ]
)
threshold_labels = np.concatenate(
    [
        np.ones_like(datasets["init_known"]["val"][1]),
        np.zeros_like(datasets["ood"][1]),
    ]
)

# scores = np.zeros((eval_datasets.shape[0], len(gmm_list)))
# for i, gmm in enumerate(gmm_list):
#     scores[:, i] = gmm.score_samples(eval_datasets)
scores = gmm_list[0].score_samples(eval_datasets)
auc = roc_auc_score(eval_labels, scores)
# scores = scores - np.min(scores)
# scores = -scores / np.max(scores)
scores_threshold = gmm_list[0].score_samples(threshold_datasets)
fprs, tprs, thresholds = roc_curve(threshold_labels, scores_threshold)
best_threshold = thresholds[np.argmax(tprs - fprs)]
preds = scores > best_threshold
acc = balanced_accuracy_score(eval_labels, preds)

print(f"Balanced Accuracy: {round(acc, 4) * 100}")
print(f"AUC: {round(auc, 4) * 100}")

Balanced Accuracy: 58.830000000000005
AUC: 91.67999999999999
